OSEMN Model (1): Obtain
SUMMARY BLAH BLAH
- Kaggle (32,000)
    - clickbait (from ‘BuzzFeed’, ‘Upworthy’, ‘ViralNova’, ‘Thatscoop’, ‘Scoopwhoop’ and ‘ViralStories’)
    - non-clickbait (from ‘WikiNews’, ’New York Times’, ‘The Guardian’, and ‘The Hindu’)
    
- Webscraping from clickbait websites ()
    - Clickhole (2800)
    - 
    - 
    
- API's from major press companies
    - News
    - 
    - 

# Kaggle Dataset

In [10]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
kaggle = pd.read_csv('Data/clickbait_data.csv')

In [3]:
kaggle.head()

,headline,clickbait
0,Should I Get Bings,1
1,Which TV Female Friend Group Do You Belong In,1
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1
3,"This Vine Of New York On ""Celebrity Big Brothe...",1
4,A Couple Did A Stunning Photo Shoot With Their...,1


Description

Dataset for Classification of news headlines into clickbait or non-clickbait.
The data is collected from various news sites.
The clickbait headlines are collected from sites such as ‘BuzzFeed’, ‘Upworthy’, ‘ViralNova’, ‘Thatscoop’, ‘Scoopwhoop’ and ‘ViralStories’.
The relevant or non-clickbait headlines are collected from many trustworthy news sites such as ‘WikiNews’, ’New York Times’, ‘The Guardian’, and ‘The Hindu’.
We can apply different classification algorithms to classify the data into clickbait and non-clickbait.

Your data will be in front of the world's largest data science community. What questions do you want to see answered?

About this file

This dataset contains headlines from various news sites such as ‘WikiNews’, ’New York Times’, ‘The Guardian’, ‘The Hindu’, ‘BuzzFeed’, ‘Upworthy’, ‘ViralNova’, ‘Thatscoop’, ‘Scoopwhoop’ and ‘ViralStories’. It has two columns first one contains headlines and the second one has numerical labels of clickbait in which 1 represents that it is clickbait and 0 represents that it is non-clickbait headline. The dataset contains total 32000 rows of which 50% are clickbait and other 50% are non-clickbait.

# Webscraping

## Clickhole

### Lifestyle

In [14]:
from bs4 import BeautifulSoup
import requests

https://clickhole.com/category/lifestyle/

In [39]:
html_page = requests.get('https://clickhole.com/category/lifestyle/page/1/') # Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser') # Pass the page contents to beautiful soup for parsing

In [40]:
soup.find_all('h2', {'class': 'post-title'})

[<h2 class="post-title">
 <a href="https://clickhole.com/5-magical-cyber-gems-that-when-printed-out-and-combined-as-one-will-arm-you-with-the-awesome-power-of-the-internet/">5 Magical Cyber Gems That, When Printed Out And Combined As One, Will Arm You With The Awesome Power Of The Internet!</a>
 </h2>, <h2 class="post-title">
 <a href="https://clickhole.com/with-great-art-comes-great-sacrifice-5-times-jacuzzi-phil-dove-head-first-into-the-jacuzzi-and-broke-his-skull/">With Great Art Comes Great Sacrifice: 5 Times Jacuzzi Phil Dove Head First Into The Jacuzzi And Broke His Skull</a>
 </h2>, <h2 class="post-title">
 <a href="https://clickhole.com/the-fact-that-these-animals-have-killed-people-does-not-make-them-any-less-adorable-they-are-beasts-and-do-not-know-better/">The Fact That These Animals Have Killed People Does Not Make Them Any Less Adorable. They Are Beasts And Do Not Know Better.</a>
 </h2>, <h2 class="post-title">
 <a href="https://clickhole.com/come-and-git-it-boys-6-hunky-

In [46]:
first_7 = soup.find_all('h2', {'class': 'post-title'})

In [47]:
first = first_7[0]

In [48]:
first.find('a').text

'5 Magical Cyber Gems That, When Printed Out And Combined As One, Will Arm You With The Awesome Power Of The Internet!'

In [55]:
headlines = []
raw = soup.find_all('h2', {'class': 'post-title'})
a = [title.find('a').text for title in raw]
headlines += a
headlines

['5 Magical Cyber Gems That, When Printed Out And Combined As One, Will Arm You With The Awesome Power Of The Internet!',
 'With Great Art Comes Great Sacrifice: 5 Times Jacuzzi Phil Dove Head First Into The Jacuzzi And Broke His Skull',
 'The Fact That These Animals Have Killed People Does Not Make Them Any Less Adorable. They Are Beasts And Do Not Know Better.',
 'Come And Git It, Boys! 6 Hunky Ranch Hands Who Could Use A Little Patricia In Their Lassos',
 'Prioritizing Self-Care: This Woman Is Enjoying Her Pedicure Too Much To Be Bothered By The Very Obvious Human Trafficking Situation Going On In The Nail Salon',
 'Sorry, This Kind Of Stuff Isn’t Really Our Thing: A Few Pretty Bad Blowjob Tips',
 '5 Terrifying Life Lessons I Learned While Teaching Badminton To The Elderly']

In [30]:
import time

In [60]:
urls = ['https://clickhole.com/category/lifestyle/page/{}/'.format(i) for i in range(1, 401)]

In [56]:
def get_headlines(urls):
    headlines = []
    for url in urls:
        html_page = requests.get(url)
        time.sleep(1)
        soup = BeautifulSoup(html_page.content, 'html.parser')
        raw = soup.find_all('h2', {'class': 'post-title'})
        headlines += [title.find('a').text for title in raw]
    return headlines

In [61]:
clickhole_lifestyle = get_headlines(urls)

In [62]:
print(len(clickhole_lifestyle))
clickhole_lifestyle

2794


['5 Magical Cyber Gems That, When Printed Out And Combined As One, Will Arm You With The Awesome Power Of The Internet!',
 'With Great Art Comes Great Sacrifice: 5 Times Jacuzzi Phil Dove Head First Into The Jacuzzi And Broke His Skull',
 'The Fact That These Animals Have Killed People Does Not Make Them Any Less Adorable. They Are Beasts And Do Not Know Better.',
 'Come And Git It, Boys! 6 Hunky Ranch Hands Who Could Use A Little Patricia In Their Lassos',
 'Prioritizing Self-Care: This Woman Is Enjoying Her Pedicure Too Much To Be Bothered By The Very Obvious Human Trafficking Situation Going On In The Nail Salon',
 'Sorry, This Kind Of Stuff Isn’t Really Our Thing: A Few Pretty Bad Blowjob Tips',
 '5 Terrifying Life Lessons I Learned While Teaching Badminton To The Elderly',
 'They Just Want To Get Through The Day: Axe Body Spray Has Released A New Undetectable Scent For Shy 8th Grade Boys Who Don’t Want To Be Noticed',
 '8 People You Correctly Thought Were Dead',
 '5 Fallback Anima

In [63]:
df_lifestyle = pd.DataFrame(clickhole_lifestyle)

In [64]:
df_lifestyle

,0
0,"5 Magical Cyber Gems That, When Printed Out An..."
1,With Great Art Comes Great Sacrifice: 5 Times ...
2,The Fact That These Animals Have Killed People...
3,"Come And Git It, Boys! 6 Hunky Ranch Hands Who..."
4,Prioritizing Self-Care: This Woman Is Enjoying...
...,...
2789,The World’s Dumbest Criminals
2790,9 Little-Known Holidays You’ll Never Want To M...
2791,7 Pricks Who Defied The Odds And Didn’t Go Int...
2792,EXCLUSIVE: A Look Inside The 2014 World Cup So...


In [57]:
first_50p = get_headlines(urls)

print(len(first_50p))
first_50p

350


['5 Magical Cyber Gems That, When Printed Out And Combined As One, Will Arm You With The Awesome Power Of The Internet!',
 'With Great Art Comes Great Sacrifice: 5 Times Jacuzzi Phil Dove Head First Into The Jacuzzi And Broke His Skull',
 'The Fact That These Animals Have Killed People Does Not Make Them Any Less Adorable. They Are Beasts And Do Not Know Better.',
 'Come And Git It, Boys! 6 Hunky Ranch Hands Who Could Use A Little Patricia In Their Lassos',
 'Prioritizing Self-Care: This Woman Is Enjoying Her Pedicure Too Much To Be Bothered By The Very Obvious Human Trafficking Situation Going On In The Nail Salon',
 'Sorry, This Kind Of Stuff Isn’t Really Our Thing: A Few Pretty Bad Blowjob Tips',
 '5 Terrifying Life Lessons I Learned While Teaching Badminton To The Elderly',
 'They Just Want To Get Through The Day: Axe Body Spray Has Released A New Undetectable Scent For Shy 8th Grade Boys Who Don’t Want To Be Noticed',
 '8 People You Correctly Thought Were Dead',
 '5 Fallback Anima

In [58]:
df = pd.DataFrame(first_50p)

In [59]:
df

,0
0,"5 Magical Cyber Gems That, When Printed Out An..."
1,With Great Art Comes Great Sacrifice: 5 Times ...
2,The Fact That These Animals Have Killed People...
3,"Come And Git It, Boys! 6 Hunky Ranch Hands Who..."
4,Prioritizing Self-Care: This Woman Is Enjoying...
...,...
345,Probably A Lost Cause: 6 Amazing Quotes That W...
346,4 Classic ‘Who Wants To Be A Millionaire’ Epis...
347,Slay Queen! 5 Times Grandma Was An Absolute Gi...
348,Efficiency Win: Heinz Has Eliminated All Their...


### asdf

## source 2

## source 3

# API

## Datanews

In [66]:
import json

In [107]:
from config import datanews_key

ImportError: cannot import name 'datanews_key'

In [104]:
datanews_key

NameError: name 'datanews_key' is not defined

In [67]:
!pip install datanews

In [ ]:
import datanews
datanews.api_key = news_key

response = datanews.headlines(q='SpaceX', language=['en'])
articles = response['hits']
print(articles[0]['title'])

## source 2

## source 3